<a href="https://colab.research.google.com/github/FuryBaM/Togyzkumalak-DQN/blob/master/Togyzkumalak_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#conv2d dqn vs minimax
#!pip show tensorflow
#!pip install --upgrade tensorflow
#!pip install pydot
#!pip install graphviz
import math, random, inspect, time, datetime, random, os, psutil, gc
import numpy as np
from timeit import default_timer as timer
from copy import deepcopy
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Multiply, Dropout, Conv1D, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, AveragePooling2D, LSTM
from keras.utils import plot_model
from collections import deque

class Player:
    def __init__(self, name="Unknown player"):
        self.score = 0
        self.name = name

class Human(Player):
    def __init__(self, name):
        Player.__init__(self, name)


class AI(Player):
    def __init__(self, name):
        Player.__init__(self, name)

    def getMove(self):
        pass

class MinimaxAI(AI):
    def __init__(self, name):
        AI.__init__(self, name)

    def getMove(self, board, depth):
        bestmove = None
        besteval = -math.inf

        bestmoves = []
        possibleMoves = board.getPossibleMoves()
        start = datetime.now()
        for i, move in enumerate(possibleMoves):
            board.makeMove(move[0], move[1])
            eval = Board.minimax(board, self, depth, -float("inf"), float("inf"), True)
            board.undoMove()
            if eval > besteval:
                besteval = eval
                bestmoves = [move]
            elif abs(eval - besteval) <= 0:
                bestmoves.append(move)
        move = random.choice(bestmoves)
        return move

class RandomAI(AI):
    def __init__(self, name):
        AI.__init__(self, name)

    def getMove(self, board):
        return random.choice(board.possibleMoves)

class DQN(AI):
    def __init__(self, name):
        AI.__init__(self, name)
        self.action_size = 9
        self.memory = deque(maxlen=2048)  # хранение опыта
        self.gamma = 0.95 # коэффициент дисконтирования
        self.epsilon = 0.0 # исследование против эксплуатации
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.9995
        self.learning_rate = 0.0001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.target_update_counter = 0
        self.update_frequency = 1e4

    def _build_model(self):
        # Входные слои для различных типов данных
        input_board = Input(shape=(2, 9, 1), name="board")
        input_tuzdyk = Input(shape=(2,), name="tuzdyk")
        input_kazan = Input(shape=(2,), name="kazan")

        #zeropad = ZeroPadding2D(padding = 2)(input_board)
        board_features = Conv2D(81, (1, 1), padding = "same", activation='linear')(input_board)
        board_features = Conv2D(162, (2, 2), padding = "same", activation='linear')(board_features)
        board_features = Conv2D(162, (2, 2), padding = "same", activation='linear')(board_features)
        #board_features = AveragePooling2D()(activation)
        flatten = Flatten()(board_features)

        merged = Concatenate()([flatten, input_tuzdyk, input_kazan])

        # Четвертый блок слоев, работающий над объединенными данными
        merged_features = Dense(162, activation="linear")(merged)
        merged_features = Dropout(0.1)(merged_features)
        merged_features = Dense(162, activation="linear")(merged_features)
        merged_features = Dense(162, activation="linear")(merged_features)
        merged_features = Dropout(0.1)(merged_features)

        # Пятый блок слоев, отвечающий за выход
        output_layer = Dense(9, activation="linear")(merged_features)

        #merged = Concatenate()([output_layer, input_kazan])

        # Седьмой блок слоев, дополнительная обработка перед финальным выходом
        final_features = Dense(81, activation="linear")(output_layer)
        final_features = Dense(81, activation="linear")(final_features)
        final_features = Dense(18, activation="linear")(final_features)


        # Финальный выходной слой
        output = Dense(self.action_size, activation="linear")(final_features)

        # Составление модели
        model = Model(inputs=[input_board, input_kazan, input_tuzdyk], outputs=output)
        model.compile(
            loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        )
        model.summary()
        plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def getState(self, board):
        tuzdyk1 = -1
        tuzdyk2 = -1
        if board.tuzdyk1 != None:
            tuzdyk1 = board.tuzdyk1[0]
        if board.tuzdyk2 != None:
            tuzdyk2 = board.tuzdyk2[0]
        input_board = [board.board[0], board.board[1]]
        kazan = np.array([board.player1.score, board.player2.score])
        tuzdyk = np.array([tuzdyk1, tuzdyk2])
        if self != board.player1:
            kazan = np.array([board.player2.score, board.player1.score])
            input_board = [board.board[1], board.board[0]]
            tuzdyk = np.array([tuzdyk2, tuzdyk1])

        input_board = np.array(input_board).reshape(1, 2, 9)
        #print(input_board)
        kazan = kazan.reshape(1, 2)
        tuzdyk = tuzdyk.reshape(1, 2)
        input_board = tf.convert_to_tensor(input_board, dtype=tf.int32)
        kazan = tf.convert_to_tensor(kazan, dtype=tf.int32)
        tuzdyk = tf.convert_to_tensor(tuzdyk, dtype=tf.int32)
        return [input_board, kazan, tuzdyk]

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def act(self, state, board):
        if np.random.rand() <= self.epsilon:# or len(board.moveHistory) <= 2:
            move = random.choice(board.possibleMoves)
            return move # случайное действие
        act_values = self.model.predict(state, verbose=0)
        #print(act_values)
        possible_moves = board.possibleMoves
        y = 0 if self == board.player1 else 1
        for i in range(self.action_size):
            if (i, y) not in possible_moves:
                act_values[0][i] = -math.inf
        return (np.argmax(act_values[0]), 0 if self == board.player1 else 1)  # лучшее действие

    def replay(self, batch_size=32):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state in minibatch:
            target = reward

            if next_state is not None:
                target = reward + self.gamma * np.max(self.target_model.predict(next_state, verbose = 0)[0])
            target_f = self.model.predict(state, verbose = 0)
            target_f[0][action] = target

            with tf.GradientTape() as tape:
                q_values = self.model(state)
                loss = tf.reduce_mean(tf.square(target_f - q_values))
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.model.optimizer.apply_gradients(zip(grads, self.model.trainable_variables), verbose = 0)
            self.target_update_counter += 1
            if self.target_update_counter >= self.update_frequency:
                print("Target network has been updated")
                self.update_target_model()
                self.target_update_counter = 0

        # Уменьшить epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def load(self, name):
        self.model = tf.keras.models.load_model(name)
        self.update_target_model()

    def save(self, name):
        self.model.save(name, overwrite = True)

    def load_weights(self, name):
        self.model.load_weights(name)
        self.update_target_model()

    def save_weights(self, name):
        self.model.save_weights(name, overwrite = True)


class Board:
    def __init__(self, player1, player2):
        self.board = [[9,9,9,9,9,9,9,9,9],
                      [9,9,9,9,9,9,9,9,9]]
        self.player1 = player1
        self.player2 = player2
        self.turnOwner = player1
        self.tuzdyk1 = None
        self.tuzdyk2 = None
        self.moveHistory = []
        self.semiTurns = 1
        self.fullTurns = 1
        self.lastMove = None
        self.possibleMoves = self.getPossibleMoves()

    def setTKFEN(self, fen):
        self.reset()
        fen_elements = fen.split()
        state = fen_elements[0].split("-")
        for i, cell in enumerate(state):
            x = i % 9
            y = i // 9
            if cell == "X":
                if y == 0:
                    self.tuzdyk2 = (x, y)
                else:
                    self.tuzdyk1 = (x, y)
                self.board[y][x] = 0
            else:
                self.board[y][x] = int(cell)
        current_turn = fen_elements[1]
        if current_turn == "w":
            self.turnOwner = self.player1
        else:
            self.turnOwner = self.player2
        self.player1.score = int(fen_elements[2])
        self.player2.score = int(fen_elements[3])
        self.semiTurns = int(fen_elements[4])
        self.fullTurns = int(fen_elements[5])
        self.possibleMoves = self.getPossibleMoves()

    def getTKFEN(self):
        board = ""
        for y, row in enumerate(self.board):
            for x, item in enumerate(row):
                if self.tuzdyk1 == (x, y) or self.tuzdyk2 == (x, y):
                    board += 'X'
                else:
                    board += str(item)
                if x != len(row) - 1:
                    board += "-"
            if y != len(self.board) - 1:
                board += "-"
        turnOwner = "w"
        if self.turnOwner == self.player2:
            turnOwner = "b"
        return f"{board} {turnOwner} {self.player1.score} {self.player2.score} {self.semiTurns} {self.fullTurns}"

    def getState(self):
        return (
            str(self.board),
            self.turnOwner == self.player1,
            (self.tuzdyk1, self.tuzdyk2),
            (self.player1.score, self.player2.score),
        )

    def reset(self):
        self.board = [[9,9,9,9,9,9,9,9,9],
                      [9,9,9,9,9,9,9,9,9]]
        self.moveHistory.clear()
        self.player1.score = self.player2.score = 0
        self.turnOwner = self.player1
        self.tuzdyk1 = self.tuzdyk2 = None
        self.semiTurns = self.fullTurns = 1
        self.possibleMoves = self.getPossibleMoves()

    def showBoard(self):
        player1Desk = "".join(
            f"{self.board[0][i] if self.tuzdyk2 != (i, 0) else 'X'} " for i in range(9)
        )
        player2Desk = "".join(
            f"{self.board[1][i] if self.tuzdyk1 != (i, 1) else 'X'} "
            for i in reversed(range(9))
        )
        print("__________________________")
        print(f"Turn: {self.semiTurns}\n Makes turn: {self.turnOwner.name}")
        print(f"{self.player2.name} Score: {self.player2.score}")
        print("".join([f"{i} " for i in reversed(range(9))]))
        print()
        print(player2Desk)
        print(player1Desk)
        print()
        print("".join([f"{i} " for i in range(9)]))
        print(f"{self.player1.name} Score: {self.player1.score}")
        print("__________________________")

    def predict(self, x, y):
        stones = self.getStoneCountAtCell(x, y)
        if stones == 0:
            return (x, y)
        elif stones == 1:
            if x == 8:
                return (0, 0 if y == 1 else 1)
            else:
                return (x + 1, y)
        else:
            stones -= 1
        maxstones = stones
        while stones > 0:
            stones = stones - (8 - x)
            x += maxstones - stones
            maxstones = stones
            if stones <= 0 and x + maxstones <= 8:
                return (x + maxstones, y)
            if x >= 8:
                x = -1
                y = 0 if y == 1 else 1
        return (x, y)

    def evaluate(self, player):
        win_factor = 0
        score_factor = player.score
        tuzdyk_factor = 0
        possible_moves_factor = self.getPossibleMovesForPlayer(player)
        desk_stones_factor = self.getPlayerDeskStones(player)

        if player == self.player1:
            winner = self.checkWinner()
            score_factor -= self.player2.score
            desk_stones_factor -= self.getPlayerDeskStones(self.player2)
            possible_moves_factor -= self.getPossibleMovesForPlayer(self.player2)
            if winner == self.player1:
                win_factor = 162 - self.player1.score
            elif winner == self.player2:
                win_factor = self.player2.score - 162
            if self.tuzdyk1 is not None:
                tuzdyk_factor += 4 - abs(self.tuzdyk1[0] - 4) * 5
            if self.tuzdyk2 is not None:
                tuzdyk_factor -= 4 - abs(self.tuzdyk2[0] - 4) * 5
        elif player == self.player2:
            winner = self.checkWinner()
            score_factor -= self.player1.score
            desk_stones_factor -= self.getPlayerDeskStones(self.player1)
            possible_moves_factor -= self.getPossibleMovesForPlayer(self.player1)
            if winner == self.player2:
                win_factor = 162 - self.player2.score
            elif winner == self.player1:
                win_factor = self.player1.score - 162
            if self.tuzdyk2 is not None:
                tuzdyk_factor += 4 - abs(self.tuzdyk2[0] - 4) * 5
            if self.tuzdyk1 is not None:
                tuzdyk_factor -= 4 - abs(self.tuzdyk1[0] - 4) * 5

        vulnerableCells, vulnerableStones = self.getVulnerables(player)
        attackableCells, attackableStones = self.getAttackables(player)
        factors = [
            score_factor,
            desk_stones_factor * 0.01,
            -vulnerableCells,
            -vulnerableStones,
            tuzdyk_factor,
            attackableCells,
            attackableStones,
            win_factor,
            possible_moves_factor
        ]

        eval = sum(factors)
        return eval

    def minimax(board, player, depth, alpha, beta, maximizing_player):
        if depth == 0 or board.checkWinner() is not None:
            return board.evaluate(player)
        if maximizing_player:
            max_eval = float('-inf')
            for move in board.possibleMoves:
                board.makeMove(move[0], move[1])  # Реализуйте функцию, делающую ход в игре
                eval = Board.minimax(board, player, depth - 1, alpha, beta, False)
                board.undoMove()
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in board.possibleMoves:
                board.makeMove(move[0], move[1])  # Реализуйте функцию, делающую ход в игре
                eval = Board.minimax(board, player, depth - 1, alpha, beta, True)
                board.undoMove()
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval


    def getPossibleMovesForPlayer(self, player):
        totalMoves = 0
        if self.turnOwner != player:
            if player == self.player1:
                for x in range(9):
                    if self.getStoneCountAtCell(x, 0) != 0:
                        totalMoves += 1
            else:
                for x in range(9):
                    if self.getStoneCountAtCell(x, 1) != 0:
                        totalMoves += 1
        else:
            totalMoves = len(self.possibleMoves)
        return totalMoves

    def getVulnerables(self, player):
        totalCells = 0
        loss = float("-inf")
        if self.turnOwner == player:
            loss = float("-inf")
            myMoves = self.possibleMoves
            for myMove in myMoves:
              self.makeMove(myMove[0], myMove[1])
              totalCells = 0
              oppMoves = self.possibleMoves
              for oppMove in oppMoves:
                attackedPosition = self.predict(oppMove[0], oppMove[1])
                attackedStones = self.getStoneCountAtCell(attackedPosition[0], attackedPosition[1])
                stonesInArm = self.getStoneCountAtCell(oppMove[0], oppMove[1]) - 1
                if oppMove[1] != attackedPosition[1] and attackedStones % 2 != 0:
                    attackedStones = attackedStones + stonesInArm // 18 + 1
                    if loss == None or attackedStones > loss:
                        loss = attackedStones
                    totalCells += 1
              self.undoMove()
        else:
          totalCells = 0
          oppMoves = self.possibleMoves
          for oppMove in oppMoves:
            attackedPosition = self.predict(oppMove[0], oppMove[1])
            attackedStones = self.getStoneCountAtCell(attackedPosition[0], attackedPosition[1])
            stonesInArm = self.getStoneCountAtCell(oppMove[0], oppMove[1]) - 1
            if oppMove[1] != attackedPosition[1] and attackedStones % 2 != 0:
                attackedStones = attackedStones + stonesInArm // 18 + 1
                if loss == None or attackedStones > loss:
                    loss = attackedStones
                totalCells += 1
        if loss == float("-inf"):
            loss = 0
        return totalCells, loss

    def getAttackables(self, player):
        captured = None
        totalCells = 0
        if player == self.turnOwner:
            for move in self.possibleMoves:
                attackedPosition = self.predict(move[0], move[1])
                attackedStones = self.getStoneCountAtCell(attackedPosition[0], attackedPosition[1])
                stonesInArm = self.getStoneCountAtCell(move[0], move[1]) - 1
                if move[1] != attackedPosition[1] and attackedStones % 2 != 0:
                    attackedStones = attackedStones + stonesInArm // 18 + 1
                    if captured == None or attackedStones > captured:
                        captured = attackedStones
                    totalCells += 1
            if captured == None:
                captured = 0
        else:
            captured = float("-inf")
            for move in self.possibleMoves:
                capturedStones = None
                cells = 0
                self.makeMove(move[0], move[1])
                for move in self.possibleMoves:
                    attackedPosition = self.predict(move[0], move[1])
                    attackedStones = self.getStoneCountAtCell(attackedPosition[0], attackedPosition[1])
                    stonesInArm = self.getStoneCountAtCell(move[0], move[1]) - 1
                    if move[1] != attackedPosition[1] and attackedStones % 2 != 0:
                        attackedStones = attackedStones + stonesInArm // 18 + 1
                        if capturedStones == None or attackedStones > capturedStones:
                            capturedStones = attackedStones
                        cells += 1
                self.undoMove()
                if capturedStones is None:
                    capturedStones = 0
                if capturedStones > captured:
                    captured = capturedStones
                    totalCells = cells
            if captured == float("-inf") or captured is None:
                captured = 0
        return totalCells, captured

    def getPlayerDeskStones(self, player):
        result = 0
        if player == self.player1:
            for i in range(9):
                result += self.getStoneCountAtCell(i, 0)
        else:
            for i in range(9):
                result += self.getStoneCountAtCell(i, 1)
        return result

    def checkWinner(self):
        if self.player1.score >= 82 or (
            self.player2.score < 82 and self.isDeskEmpty(self.player2)
        ):
            return self.player1
        elif self.player2.score >= 82 or (
            self.player1.score < 82 and self.isDeskEmpty(self.player1)
        ):
            return self.player2
        return None

    def isDeskEmpty(self, player):
        index = 0
        if player == self.player2:
            index = 1
        for i in range(9):
            if self.board[index][i] > 0:
                return False
        return True

    def isValidMove(self, x, y):
        return (
            (self.turnOwner == self.player1 and y == 0)
            or (self.turnOwner == self.player2 and y == 1)
        ) and self.isCellEmpty(x, y) == False

    def isCellEmpty(self, x, y):
        return self.board[y][x] == 0

    def getStoneCountAtCell(self, x, y):
        return self.board[y][x]

    def getPossibleMoves(self):
        possibleMoves = []
        if self.checkWinner() is not None:
            return possibleMoves
        if self.turnOwner == self.player1:
            for x in range(9):
                if not self.isCellEmpty(x, 0):
                    possibleMoves.append((x, 0))
        else:
            for x in range(9):
                if not self.isCellEmpty(x, 1):
                    possibleMoves.append((x, 1))
        return possibleMoves

    def switchPlayer(self):
        if self.turnOwner == self.player1:
            self.turnOwner = self.player2
        else:
            self.fullTurns += 1
            self.turnOwner = self.player1

    def makeMove(self, x, y):
        stonesInArm = 0
        if not self.isValidMove(x, y) or self.checkWinner() != None:
            return False
        self.lastMove = ""
        self.lastMove = f"{x + 1}"
        self.semiTurns += 1
        self.moveHistory.append((
                Board.copyBoard(self.board),
                self.turnOwner,
                self.tuzdyk1,
                self.tuzdyk2,
                self.player1.score,
                self.player2.score))
        if self.getStoneCountAtCell(x, y) > 1:
            stonesInArm = self.getStoneCountAtCell(x, y) - 1
            self.board[y][x] = 1
        else:
            stonesInArm = 1
            self.board[y][x] = 0
        position = (x, y)
        for _ in range(stonesInArm):
            position = ((position[0] + 1) % 9, 1 - position[1] if position[0] == 8 else position[1])

            if position == self.tuzdyk1:
                self.player1.score += 1
            elif position == self.tuzdyk2:
                self.player2.score += 1
            else:
                self.board[position[1]][position[0]] += 1

        x, y = position
        if self.board[y][x] == 3:  # make tuzdyk
            if (self.turnOwner == self.player1 and x != 8 and y == 1 and self.tuzdyk1 == None):
                if (self.tuzdyk2 != None and self.tuzdyk2[0] != x) or self.tuzdyk2 == None:
                    self.tuzdyk1 = (x, y)
                    self.board[y][x] = 0  # player 1
                    self.player1.score += 3
            elif (self.turnOwner == self.player2 and x != 8 and y == 0 and self.tuzdyk2 == None):
                if (self.tuzdyk1 != None and self.tuzdyk1[0] != x) or self.tuzdyk1 == None:
                    self.tuzdyk2 = (x, y)
                    self.board[y][x] = 0  # player 2
                    self.player2.score += 3
        if self.board[y][x] % 2 == 0:  # grab stones
            if (self.turnOwner == self.player1 and y == 1) or (self.turnOwner == self.player2 and y == 0):
                self.turnOwner.score += self.board[y][x]
                self.board[y][x] = 0
        self.switchPlayer()
        self.lastMove += f"{x+1}"
        self.possibleMoves = self.getPossibleMoves()
        return True

    def undoMove(self):
        if len(self.moveHistory) == 0:
            return False
        lastMove = self.moveHistory.pop()
        self.board = lastMove[0]
        self.turnOwner = lastMove[1]
        self.tuzdyk1 = lastMove[2]
        self.tuzdyk2 = lastMove[3]
        self.player1.score = lastMove[4]
        self.player2.score = lastMove[5]
        self.semiTurns -= 1
        self.possibleMoves = self.getPossibleMoves()
        if self.turnOwner == self.player2:
            self.fullTurns -= 1
        return True

    def copyBoard(boardArray):
        return [row[:] for row in boardArray]

player1 = DQN("DQN-1")
player2 = MinimaxAI("Minimax")
#player1.load("DQN_convmodelD2")
#player1.load_weights("/content/drive/MyDrive/Togyzkumalak/DQN_convmodel")
#player1.load("/kaggle/input/togyzkumalakdb/MODELMCTS")
board = Board(player1, player2)

# Параметры обучения
n_episodes = 10000  # количество эпизодов
batch_size = 32  # размер пакета для обучения
white_wins = 0
black_wins = 0
rewards = []
last_checking_rewards = []
last_rewards_len = 16
start = timer()
for episode in range(n_episodes):
    state = None
    done = False
    myMove = None
    oppMove = None
    reward = 0
    total_reward = 0
    game_rewards = []
    if episode % 25 == 0:
        print("Target update counter:", player1.target_update_counter, "/", player1.update_frequency)
        print("Saving...")
        player1.save("/content/drive/MyDrive/Togyzkumalak/DQN_convmodel")
        player1.save_weights("/content/drive/MyDrive/Togyzkumalak/DQN_convmodel")
        print(f"Model has saved at {episode}")
        ram_info = psutil.virtual_memory()
        print(f"Total: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
        print(f"Available: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Used: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
        print(f"Percentage usage: {ram_info.percent}%")
        gc.collect()
        tf.keras.backend.clear_session()
    while done == False:
        #board.showBoard()
        #print(board.evaluate(player1), board.evaluate(player2))
        if board.checkWinner() is not None:
            if board.checkWinner() == board.player1:
                white_wins += 1
            elif board.checkWinner() == board.player2:
                black_wins += 1
            if len(player1.memory) // 2 < batch_size:
                player1.replay(len(player1.memory) // 2)
            else:
                player1.replay(batch_size)
            curr_time = time.strftime("%H:%M:%S", time.localtime())
            print(f"[{curr_time}] Episode: {episode+1}/{n_episodes}, Wins: {white_wins} - {black_wins}, Scores: {board.player1.score} - {board.player2.score}, Epsilon: {player1.epsilon}, Total reward: {total_reward}, Turns: {board.fullTurns}\nWinrate: {white_wins/(episode + 1)}")
            board.reset()
            print("memory size:", len(player1.memory))
            break
        if board.turnOwner == player1:
            state = player1.getState(board)
            myMove = player1.act(state, board)
            reward = Board.minimax(board, player1, 2, float("-inf"), float("inf"), True)
            board.makeMove(myMove[0], myMove[1])
        else:
            oppMove = player2.getMove(board, 1)
            board.makeMove(oppMove[0], oppMove[1])
            if myMove != None:
                reward = (Board.minimax(board, player1, 2, float("-inf"), float("inf"), True) - reward) * 0.01
                next_state = player1.getState(board)
                #print(reward)
                total_reward += reward
                game_rewards.append(reward)
            if myMove!=None:
                player1.remember(state, myMove[0], reward, next_state)
    rewards.append(total_reward)
    last_checking_rewards.append(sum(game_rewards[-last_rewards_len:]))
end = timer()
td = timedelta(seconds = int(end - start))
print(f"Elapsed time, {td}")

tf.keras.backend.clear_session()
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize = (120,30), dpi = 100)
episodes = np.arange(1, len(rewards) + 1)
plt.plot(episodes, rewards, label = "Total rewards")
plt.plot(episodes, last_checking_rewards, label = "Stepwise last 16 rewards")
plt.title('Togyz kumalak')
plt.xlabel('Episode')
plt.ylabel('Rewards')
plt.legend(loc="lower right")
plt.savefig('график.png')
plt.show()
print("Average reward", sum(rewards)/n_episodes)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 board (InputLayer)          [(None, 2, 9, 1)]            0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 2, 9, 81)             162       ['board[0][0]']               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 2, 9, 162)            52650     ['conv2d[0][0]']              
                                                                                                  
 conv2d_2 (Conv2D)           (None, 2, 9, 162)            105138    ['conv2d_1[0][0]']            
                                                                                              

[11:36:09] Episode: 1/10000, Wins: 0 - 1, Scores: 17 - 88, Epsilon: 0.0, Total reward: -1.5713, Turns: 13
Winrate: 0.0
memory size: 12
[11:36:13] Episode: 2/10000, Wins: 0 - 2, Scores: 12 - 103, Epsilon: 0.0, Total reward: -1.4601000000000002, Turns: 16
Winrate: 0.0
memory size: 27
[11:36:19] Episode: 3/10000, Wins: 0 - 3, Scores: 37 - 82, Epsilon: 0.0, Total reward: -1.3695, Turns: 23
Winrate: 0.0
memory size: 49
[11:36:26] Episode: 4/10000, Wins: 0 - 4, Scores: 30 - 93, Epsilon: 0.0, Total reward: -1.2711000000000001, Turns: 15
Winrate: 0.0
memory size: 63
[11:36:33] Episode: 5/10000, Wins: 0 - 5, Scores: 44 - 82, Epsilon: 0.0, Total reward: -1.2398, Turns: 32
Winrate: 0.0
memory size: 94
[11:36:43] Episode: 6/10000, Wins: 0 - 6, Scores: 54 - 82, Epsilon: 0.0, Total reward: -0.9688000000000001, Turns: 30
Winrate: 0.0
memory size: 123
[11:36:50] Episode: 7/10000, Wins: 0 - 7, Scores: 12 - 88, Epsilon: 0.0, Total reward: -1.4699999999999998, Turns: 18
Winrate: 0.0
memory size: 140
[11:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/Colab Notebooks/Togyzkumalak DQN')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
